In [ ]:
# default_exp core

# Core library

> Helper functions used throughout the lessons.

In [ ]:
# export
from nbdev.showdoc import *
import wget
import os

## Datasets

In [ ]:
# exports
def get_dataset(dataset_name: str):
    """
    Download datasets from Google Drive.
    """

    name_to_id = {
        "word2vec-google-news-300.pkl": "1dRwSXbFTcQbn8c3V24G92wFY4DXZ1SDt",
        "imdb.csv": "1wF0YEmQOwceJz2d6w4CfhBgydU87dPGl",
        "housing.csv": "1d7oOKdDmZFx8wf0c8OfuTW1FpUyJHABh",
        "housing_gmaps_data_raw.csv": "1R1RUHAXxzrIngRJMFwyp4vZRVICd-I6T",
        "housing_addresses.csv": "1mOK0uyRz5Zs-Qo7mVMlxwtb2xn1E6N9Q",
        "housing_merged.csv": "1bdYuBtIPrKiU-ut2MeSSsL47onPtZrRt"
    }

    path = '../data/'
    gdrive_path = "https://docs.google.com/uc?export=download&id="
    if dataset_name in name_to_id:
        if os.path.exists(path + dataset_name):
            print(f"Dataset already exists at '{path + dataset_name}' and is not downloaded again.")
            return
        try:
            file_url =  gdrive_path + name_to_id[dataset_name]
            wget.download(file_url, out=path)
        except Exception as e:
            print("Something went wrong during download. Try again.")
            raise e
        print(f"Download of {dataset_name} dataset complete.")
    else:
        raise KeyError("File not on Google Drive.")

### California Housing Prices
This dataset from Kaggle ([link](https://www.kaggle.com/camnugent/california-housing-prices)) is used in the second chapter of Aurélien Géron's recent book *Hands-On Machine learning with Scikit-Learn and TensorFlow*.

#### `housing.csv`
This dataset pertains to the houses found in a given California district and some summary stats about them based on the 1990 census data.

In [ ]:
get_dataset('housing.csv')

Download of housing.csv dataset complete.


#### `housing_gmaps_data_raw.csv`
This dataset contains the raw outputs of the addresses associated with the coordinates in the `housing.csv` dataset as retrieved with the Google Maps API. 

In [ ]:
get_dataset('housing_gmaps_data_raw.csv')

Download of housing_gmaps_data_raw.csv dataset complete.


#### `housing_addresses.csv`
The `housing_addresses.csv` dataset is a cleaned subset of the `housing_gmaps_data_raw.csv` dataset.

In [ ]:
get_dataset('housing_addresses.csv')

Download of housing_addresses.csv dataset complete.


#### housing_merged.csv

The merge of `housing.csv` and `housing_addresses.csv` from lesson 2.

In [ ]:
get_dataset('housing_merged.csv')

Download of housing_merged.csv dataset complete.


### IMDB

#### `imdb.csv`
The IMDB dataset is available on Kaggle ([link](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)). This is a dataset for binary sentiment classification and provides a set of 25,000 highly polar movie reviews for training and 25,000 for testing.

In [ ]:
get_dataset('imdb.csv')

Download of imdb.csv dataset complete.


### Word2Vec embeddings

#### `word2vec-google-news-300.pkl`

Pre-trained vectors trained on a part of the Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. The phrases were obtained using a simple data-driven approach described in *Distributed Representations of Words and Phrases and their Compositionality*. This dataset is available from GENSIM ([link](https://github.com/RaRe-Technologies/gensim-data)).

In [ ]:
get_dataset('word2vec-google-news-300.pkl')

Download of word2vec-google-news-300.pkl dataset complete.
